# Environment

In [1]:
import os

# Set the paths
DATASET_HOME_DIR = '/content/drive/MyDrive/DLProj/Dataset'
CHECKPOINTS_DIR = '/content/drive/MyDrive/DLProj/vqvae_openai/checkpoints'
WAV_DATASET_DIR = os.path.join(DATASET_HOME_DIR, "WavVersion/train_small_files")  # the Slackh2100 dataset with .wav stems
REPO_DIR = '/content/masked-lm-audio'
JUKEBOX_DIR = f'{REPO_DIR}/jukebox'

# Clone the repo
!git clone https://github.com/matpaolacci/masked-lm-audio.git --quiet

# Training

In [ ]:
!mkdir train_data checkpoints
os.chdir(REPO_DIR)

#!mkdir checkpoints train_data
!unzip -j $DATASET_HOME_DIR/encoded_level_1.zip -d /content/train_data
#!unzip -j /content/archivio.zip -d /content/train_data # sposta archivio.zip prima
!mkdir checkpoints
os.chdir(REPO_DIR)

In [3]:
# update repository
!git -C $REPO_DIR pull origin main

From https://github.com/matpaolacci/masked-lm-audio
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
# Build the vocabulary
!python $REPO_DIR/bert/dataset/vocab.py -c ../train_data -o ../vocab.bin

Building Vocab
VOCAB SIZE: 2053


# Train the model!

In [ ]:
!python -m bert \
-v ../vocab.bin \
-o ../checkpoints \
-c ../train_data \
--shuffle_train_dataset True \
--num_workers 2 \
--seq_len 512 \
--batch_size 32 \
--log_freq 42\
--lr 1e-4 \
--epochs 4000 \
--hidden 512 \
--attn_heads 8 \
--layers 4 \
--random_seed 42 \
--schedule_optim_warmup_steps 6000 \
--mode train

# Evaluate

In [5]:
# Put a .pt file into eval_data from train_data
!mkdir ../checkpoints ../eval_data

mkdir: cannot create directory ‘../checkpoints’: File exists


In [14]:
!python -m bert \
-v ../vocab.bin \
--seq_len 512 \
--batch_size 32 \
--hidden 512 \
--attn_heads 8 \
--layers 4 \
--mode inference \
--elements_to_mask 128 \
--max_dataset_elements 2336 \
--path_to_eval_dataset ../eval_data \
--path_to_saved_model ../checkpoints/bert_L6.6482667_.ep4 \
--path_to_save_output ../bert.output.json

Loading Vocab ../vocab.bin
Vocab Size:  2053
Loading Evaluation Dataset ../eval_data
Creating the dataset
100% 1/1 [00:00<00:00, 176.14it/s]
Created a dataset of 470 elements
Dataset successfully created!
Loading model checkpoint...
Evaluating: 100% 15/15 [00:03<00:00,  4.25it/s]
Special tokens found and removed: [0]
Input sequence length (239360) corresponds with sequence output from BERT
Average Loss: 5.936921580632528
Saving output at '../bert.output.json'
The BERT's output sequence is 0.0006057820855614974% similar to the original
